In [1]:
import logging
# Suppress PyPSA INFO messages (keep warnings and errors)
logging.getLogger('pypsa').setLevel(logging.WARNING)

import pypsa
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces

import gc
import psutil
import matplotlib.pyplot as plt

import neptune

from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

import random

import os
#import ipdb

/Users/antoniagrindrod/miniconda3/envs/pypsa-earth-rl/lib/python3.11/site-packages/swagger_spec_validator/validator12.py:18: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature request (on referencing) if you are missing an API for the kind of customization you need.
  from jsonschema import RefResolver
/Users/antoniagrindrod/miniconda3/envs/pypsa-earth-rl/lib/python3.11/site-packages/swagger_spec_validator/validator12.py:18: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature re

In [2]:
def fix_artificial_lines_reasonable(network):
    """
    Fix artificial lines with reasonable capacity values:
    - s_nom = based on connected bus demand (with safety factor)
    - s_nom_extendable = False (non-extendable)
    - Keep capacity high enough to meet demand
    """
    print("=== FIXING ARTIFICIAL LINES WITH REASONABLE CAPACITY ===")

    # Find artificial lines
    artificial_lines = [line for line in network.lines.index
                       if any(keyword in str(line).lower() for keyword in ['new', '<->', 'artificial'])]

    if not artificial_lines:
        # If no artificial lines found by name, look for lines with s_nom=0
        # which is often a sign of artificial lines
        zero_capacity_lines = network.lines[network.lines.s_nom == 0].index.tolist()
        if zero_capacity_lines:
            artificial_lines = zero_capacity_lines

    print(f"Found {len(artificial_lines)} artificial lines to fix:")

    # Get maximum demand per bus across all snapshots
    bus_max_demand = {}
    for bus in network.buses.index:
        bus_demand = 0
        for load_name, load in network.loads.iterrows():
            if load.bus == bus and load_name in network.loads_t.p_set.columns:
                bus_demand = max(bus_demand, network.loads_t.p_set[load_name].max())
        bus_max_demand[bus] = bus_demand

    # Fix each artificial line with reasonable capacity
    for line_name in artificial_lines:
        # Get connected buses
        bus0 = network.lines.loc[line_name, 'bus0']
        bus1 = network.lines.loc[line_name, 'bus1']

        # Get maximum demand at these buses
        bus0_demand = bus_max_demand.get(bus0, 0)
        bus1_demand = bus_max_demand.get(bus1, 0)

        # Calculate required capacity with safety factor
        # Use 3x the higher demand to ensure adequate capacity
        safety_factor = 3.0
        required_capacity = max(bus0_demand, bus1_demand) * safety_factor

        # Ensure minimum reasonable capacity (1000 MW)
        required_capacity = max(required_capacity, 1000)

        print(f"\n Fixing: {line_name}")
        print(f"    Connected buses: {bus0} ↔ {bus1}")
        print(f"    Bus demands: {bus0}: {bus0_demand:.1f} MW, {bus1}: {bus1_demand:.1f} MW")

        # Set s_nom to required capacity
        old_s_nom = network.lines.loc[line_name, 's_nom']
        network.lines.loc[line_name, 's_nom'] = required_capacity
        print(f"    s_nom: {old_s_nom} → {required_capacity:.1f} MW")

        # Make sure line is not extendable
        if 's_nom_extendable' not in network.lines.columns:
            network.lines['s_nom_extendable'] = False
        network.lines.loc[line_name, 's_nom_extendable'] = False
        print(f"    s_nom_extendable: → False")

    return network

def remove_offshore_wind(network):
    """
    Remove offshore wind generators. 
    All of these have zero nominal capacity (likely missing data). 
    Need to remove them to avoid division by zero error in constraint check for slack gens.
    Problem is still feasible without offwind slack since pypsa optimize still feasible.
    """
    
    # First, identify offshore wind generators
    offwind_gens = network.generators[
        network.generators.index.str.contains('offwind', case=False, na=False)
    ].index
    
    print(f"Found {len(offwind_gens)} offshore wind generators:")
    print(offwind_gens.tolist())
    
    # Check their properties
    offwind_data = network.generators.loc[offwind_gens, ['p_nom', 'control', 'carrier']]
    print("\nOffshore wind generator details:")
    print(offwind_data)
    
    # Remove offshore wind generators one by one
    print(f"\nRemoving {len(offwind_gens)} offshore wind generators...")
    for gen in offwind_gens:
        network.remove("Generator", gen)

def create_pypsa_network(network_file):
    """Create a PyPSA network from the .nc file."""
    # Initialize network
    network = pypsa.Network(network_file)
    for storage_name in network.storage_units.index:
        # Use .loc for direct assignment to avoid SettingWithCopyWarning
        network.storage_units.loc[storage_name, 'cyclic_state_of_charge'] = False

        # Set marginal_cost to 0.01
        network.storage_units.loc[storage_name, 'marginal_cost'] = 0.01

        # Set marginal_cost_storage to 0.01
        network.storage_units.loc[storage_name, 'marginal_cost_storage'] = 0.01

        # Set spill_cost to 0.1
        network.storage_units.loc[storage_name, 'spill_cost'] = 0.1

        network.storage_units.loc[storage_name, 'efficiency_store'] = 0.866025 #use phs efficiency (hydro didnt have an efficiency, but i want to model them all as the same)

        # Fix unrealistic max_hours values
        current_max_hours = network.storage_units.loc[storage_name, 'max_hours']

        if 'PHS' in storage_name:
            # PHS with missing data - set to typical range
            network.storage_units.loc[storage_name, 'max_hours'] = 8.0
            print(f"Fixed {storage_name}: set max_hours to 8.0")

        elif 'hydro' in storage_name:
            # Hydro with unrealistic data - set to validated range
            network.storage_units.loc[storage_name, 'max_hours'] = 6.0
            print(f"Fixed {storage_name}: corrected max_hours from {current_max_hours} to 6.0")


    fix_artificial_lines_reasonable(network)
    remove_offshore_wind(network)

    return network

In [3]:
network = create_pypsa_network("elec_s_10_ec_lc1.0_1h.nc")

Fixed ZA0 0 PHS: set max_hours to 8.0
Fixed ZA0 5 PHS: set max_hours to 8.0
Fixed ZA0 6 hydro: corrected max_hours from 3831.6270020496813 to 6.0
=== FIXING ARTIFICIAL LINES WITH REASONABLE CAPACITY ===
Found 3 artificial lines to fix:

 Fixing: lines new ZA0 4 <-> ZA2 0 AC
    Connected buses: ZA0 4 ↔ ZA2 0
    Bus demands: ZA0 4: 15945.8 MW, ZA2 0: 452.6 MW
    s_nom: 0.0 → 47837.3 MW
    s_nom_extendable: → False

 Fixing: lines new ZA0 0 <-> ZA1 0 AC
    Connected buses: ZA0 0 ↔ ZA1 0
    Bus demands: ZA0 0: 3513.0 MW, ZA1 0: 1386.9 MW
    s_nom: 0.0 → 10538.9 MW
    s_nom_extendable: → False

 Fixing: lines new ZA0 0 <-> ZA3 0 AC
    Connected buses: ZA0 0 ↔ ZA3 0
    Bus demands: ZA0 0: 3513.0 MW, ZA3 0: 721.1 MW
    s_nom: 0.0 → 10538.9 MW
    s_nom_extendable: → False
Found 12 offshore wind generators:
['ZA0 1 offwind-ac', 'ZA0 1 offwind-dc', 'ZA0 5 offwind-ac', 'ZA0 5 offwind-dc', 'ZA0 7 offwind-ac', 'ZA0 7 offwind-dc', 'ZA0 8 offwind-ac', 'ZA0 8 offwind-dc', 'ZA1 0 offwind-ac

In [4]:
# Get all storage units
storage_units = network.storage_units

print("Storage Unit SOC Bounds Analysis")
print("=" * 50)
print(f"Number of storage units: {len(storage_units)}")
print()

if len(storage_units) > 0:
    # Check basic storage unit parameters
    print("Storage Unit Parameters:")
    print("-" * 30)
    for storage_name in storage_units.index:
        storage_unit = storage_units.loc[storage_name]
        
        # Calculate SOC bounds
        p_nom = storage_unit['p_nom']
        max_hours = storage_unit['max_hours']
        soc_max = p_nom * max_hours  # Maximum energy capacity
        soc_min = 0.0  # Typically 0 in PyPSA
        
        print(f"Storage Unit: {storage_name}")
        print(f"  p_nom: {p_nom:.2f} MW")
        print(f"  max_hours: {max_hours:.2f} hours")
        print(f"  SOC_min: {soc_min:.2f} MWh")
        print(f"  SOC_max: {soc_max:.2f} MWh")
        print(f"  Initial SOC: {storage_unit.get('state_of_charge_initial', 0):.2f} MWh")
        
        # Check if there are any non-standard SOC limits
        if 'soc_min_pu' in storage_unit.index:
            print(f"  SOC_min_pu: {storage_unit['soc_min_pu']}")
        if 'soc_max_pu' in storage_unit.index:
            print(f"  SOC_max_pu: {storage_unit['soc_max_pu']}")
            
        # Check efficiency parameters
        print(f"  Efficiency store: {storage_unit.get('efficiency_store', 1.0):.3f}")
        print(f"  Efficiency dispatch: {storage_unit.get('efficiency_dispatch', 1.0):.3f}")
        print(f"  Standing loss: {storage_unit.get('standing_loss', 0.0):.6f}")
        print()

    # Check if there are time-varying inflows
    print("Time-varying Parameters:")
    print("-" * 25)
    if hasattr(network.storage_units_t, 'inflow'):
        print("Inflow data exists:")
        inflow_data = network.storage_units_t.inflow
        for storage_name in storage_units.index:
            if storage_name in inflow_data.columns:
                inflow_series = inflow_data[storage_name]
                print(f"  {storage_name}: min={inflow_series.min():.2f}, max={inflow_series.max():.2f}, mean={inflow_series.mean():.2f} MW")
            else:
                print(f"  {storage_name}: No inflow data")
    else:
        print("No inflow data found")
    print()

    # Check snapshot weightings that affect SOC updates
    print("Snapshot Information:")
    print("-" * 20)
    print(f"Total snapshots: {len(network.snapshots)}")
    if hasattr(network, 'snapshot_weightings'):
        if hasattr(network.snapshot_weightings, 'stores'):
            delta_t_values = network.snapshot_weightings.stores
            print(f"Storage time steps (delta_t): min={delta_t_values.min():.3f}, max={delta_t_values.max():.3f}, mean={delta_t_values.mean():.3f}")
        else:
            delta_t_values = network.snapshot_weightings
            print(f"General time steps: min={delta_t_values.min():.3f}, max={delta_t_values.max():.3f}, mean={delta_t_values.mean():.3f}")
    print()

    # Additional analysis: Check which storage unit might violate SOC_min
    print("Potential SOC Violation Analysis:")
    print("-" * 35)
    for storage_name in storage_units.index:
        storage_unit = storage_units.loc[storage_name]
        initial_soc = storage_unit.get('state_of_charge_initial', 0)
        p_nom = storage_unit['p_nom']
        
        # Check if initial SOC is already at or near zero
        if initial_soc <= 0.1:  # Within 0.1 MWh of zero
            print(f"  {storage_name}: Initial SOC very low ({initial_soc:.3f} MWh) - likely to violate SOC_min")
        
        # Check standing losses
        standing_loss = storage_unit.get('standing_loss', 0.0)
        if standing_loss > 0:
            print(f"  {storage_name}: Has standing losses ({standing_loss:.6f}) - could lead to SOC violations")
            
        # Check efficiency losses
        eff_dispatch = storage_unit.get('efficiency_dispatch', 1.0)
        if eff_dispatch < 1.0:
            print(f"  {storage_name}: Dispatch efficiency < 1.0 ({eff_dispatch:.3f}) - energy lost during discharge")

else:
    print("No storage units found in the network!")

# Also check the overall network structure
print("\nNetwork Overview:")
print("=" * 20)
print(f"Generators: {len(network.generators)}")
print(f"Loads: {len(network.loads)}")
print(f"Lines: {len(network.lines)}")
print(f"Storage units: {len(network.storage_units)}")
print(f"Snapshots: {len(network.snapshots)}")

Storage Unit SOC Bounds Analysis
Number of storage units: 3

Storage Unit Parameters:
------------------------------
Storage Unit: ZA0 0 PHS
  p_nom: 2332.00 MW
  max_hours: 8.00 hours
  SOC_min: 0.00 MWh
  SOC_max: 18656.00 MWh
  Initial SOC: 0.00 MWh
  Efficiency store: 0.866
  Efficiency dispatch: 0.866
  Standing loss: 0.000000

Storage Unit: ZA0 5 PHS
  p_nom: 560.00 MW
  max_hours: 8.00 hours
  SOC_min: 0.00 MWh
  SOC_max: 4480.00 MWh
  Initial SOC: 0.00 MWh
  Efficiency store: 0.866
  Efficiency dispatch: 0.866
  Standing loss: 0.000000

Storage Unit: ZA0 6 hydro
  p_nom: 600.00 MW
  max_hours: 6.00 hours
  SOC_min: 0.00 MWh
  SOC_max: 3600.00 MWh
  Initial SOC: 0.00 MWh
  Efficiency store: 0.866
  Efficiency dispatch: 0.900
  Standing loss: 0.000000

Time-varying Parameters:
-------------------------
Inflow data exists:
  ZA0 0 PHS: No inflow data
  ZA0 5 PHS: No inflow data
  ZA0 6 hydro: min=1.91, max=1439.78, mean=43.96 MW

Snapshot Information:
--------------------
Total sn